# Practice 6: Writing Better Functions

## Decorator Guide 


### Decorator Basics

#### Python's functions are objects

In [3]:
def shout(word='yes'):
    return word.capitalize() + '!'

print(shout())
# outputs : 'Yes!'

# As an object, you can assign the function to a variable like any
# other object 

scream = shout

# Notice we don’t use parentheses: we are not calling the function, we are
# putting the function `shout` into the variable `scream`. 
# It means you can then call `shout` from `scream`:

print(scream())
# outputs : 'Yes!'

# More than that, it means you can remove the old name `shout`, and
# the function will still be accessible from `scream`

del shout
try:
    print(shout())
except NameError as e:
    print(e)
    #outputs: "name 'shout' is not defined"

print(scream())
# outputs: 'Yes!'

Yes!
Yes!
name 'shout' is not defined
Yes!


#### Python functions can be defined inside another function!

In [4]:
def talk():

    # You can define a function on the fly in `talk` ...
    def whisper(word='yes'):
        return word.lower() + '...'

    # ... and use it right away!

    print(whisper())

# You call `talk`, that defines `whisper` EVERY TIME you call it, then
# `whisper` is called in `talk`. 
talk()
# outputs: 
# "yes..."

# But `whisper` DOES NOT EXIST outside `talk`:

try:
    print(whisper())
except NameError as e:
    print(e)
    #outputs : "name 'whisper' is not defined"*

yes...
name 'whisper' is not defined


#### Functions references
You’ve seen that functions are objects. Therefore, functions:

    can be assigned to a variable
    can be defined in another function

That means that a function can return another function.

In [5]:
def getTalk(kind='shout'):

    # We define functions on the fly
    def shout(word='yes'):
        return word.capitalize() + '!'

    def whisper(word='yes'):
        return word.lower() + '...'

    # Then we return one of them
    if kind == 'shout':
        # We don’t use '()'. We are not calling the function;
        # instead, we’re returning the function object
        return shout  
    else:
        return whisper

# How do you use this strange beast?

# Get the function and assign it to a variable
talk = getTalk()      

# You can see that `talk` is here a function object: 
#함수를 변수로 지정하면 calloing function 해줘야한다(괄호 필요).
print(talk)
#outputs : <function shout at 0xb7ea817c>

<function getTalk.<locals>.shout at 0x0000021DDCE923E0>


In [6]:
# The object is the one returned by the function:
print(talk())
#outputs : Yes!

Yes!


In [7]:
################################## TODO ##################################
# And you can even use it directly if you feel wild: ???
print(getTalk('whisper')())
#outputs : yes...

yes...


If you can return a function, you can pass one as a parameter

In [8]:
def doSomethingBefore(func): 
    print('I do something before then I call the function you gave me')
    print(func())

In [9]:
################################## TODO ##################################
doSomethingBefore(scream)

#outputs: 
#I do something before then I call the function you gave me
#Yes!

I do something before then I call the function you gave me
Yes!


Well, you just have everything needed to understand decorators. You see, decorators are “wrappers”, which means that they let you execute code before and after the function they decorate without modifying the function itself.

#### Handcrafted decorators

In [10]:
# A decorator is a function that expects ANOTHER function as parameter
def my_shiny_new_decorator(a_function_to_decorate):

    # Inside, the decorator defines a function on the fly: the wrapper.
    # This function is going to be wrapped around the original function
    # so it can execute code before and after it.
    def the_wrapper_around_the_original_function():

        # Put here the code you want to be executed BEFORE the original 
        # function is called
        print('Before the function runs')

        # Call the function here (using parentheses)
        a_function_to_decorate()

        # Put here the code you want to be executed AFTER the original 
        # function is called
        print('After the function runs')

    # At this point, `a_function_to_decorate` HAS NEVER BEEN EXECUTED.
    # We return the wrapper function we have just created.
    # The wrapper contains the function and the code to execute before
    # and after. It’s ready to use!
    return the_wrapper_around_the_original_function

# Now imagine you create a function you don’t want to ever touch again.
def a_stand_alone_function():
    print('I am a stand alone function, don’t you dare modify me')

a_stand_alone_function() 
#outputs: I am a stand alone function, don't you dare modify me

# Well, you can decorate it to extend its behavior.
# Just pass it to the decorator, it will wrap it dynamically in 
# any code you want and return you a new function ready to be used:

I am a stand alone function, don’t you dare modify me


Now, you probably want that every time you call a_stand_alone_function, a_stand_alone_function_decorated is called instead. That’s easy, just overwrite a_stand_alone_function with the function returned by my_shiny_new_decorator:

In [11]:
################################## TODO ##################################
my_shiny_new_decorator(a_stand_alone_function)()

#outputs:
#Before the function runs
#I am a stand alone function, don’t you dare modify me
#After the function runs

# And guess what? That’s EXACTLY what decorators do!

Before the function runs
I am a stand alone function, don’t you dare modify me
After the function runs


#### Decorators demystified

In [12]:
@my_shiny_new_decorator
def another_stand_alone_function():
    print('Leave me alone')

another_stand_alone_function()  
#outputs:  
#Before the function runs
#Leave me alone
#After the function runs

Before the function runs
Leave me alone
After the function runs


@decorator is just a shortcut to:

In [13]:
another_stand_alone_function = my_shiny_new_decorator(another_stand_alone_function)

Decorators are just a pythonic variant of the decorator design pattern. There are several classic design patterns embedded in Python to ease development (like iterators).

Of course, you can accumulate decorators:

In [14]:
def bread(func):
    def wrapper():
        print("</''''''\>")
        func()
        print("<\______/>")
    return wrapper

def ingredients(func):
    def wrapper():
        print('#tomatoes#')
        func()
        print('~salad~')
    return wrapper

def sandwich(food='--ham--'):
    print(food)

sandwich()
#outputs: --ham--

--ham--


In [15]:
sandwich = bread(ingredients(sandwich))
sandwich()

#outputs:
#</''''''\>
# #tomatoes#
# --ham--
# ~salad~
#<\______/>

</''''''\>
#tomatoes#
--ham--
~salad~
<\______/>


Using the Python decorator syntax. Redefine function sandwich together with decorators.

In [16]:
################################## TODO ##################################
@bread
@ingredients
def sandwich(food='--ham--'):
    print(food)
    
sandwich()
    
#outputs:
#</''''''\>
# #tomatoes#
# --ham--
# ~salad~
#<\______/>

</''''''\>
#tomatoes#
--ham--
~salad~
<\______/>


The order you set the decorators MATTERS:

In [17]:
################################## TODO ##################################
# Implement function strange_sandwich that returns the following
@ingredients
@bread
def sandwich(food='--ham--'):
    print(food)
    
sandwich()
#outputs:
##tomatoes#
#</''''''\>
# --ham--
#<\______/>
# ~salad~

#tomatoes#
</''''''\>
--ham--
<\______/>
~salad~


### Taking decorators to the next level

#### Passing arguments to the decorated function

In [18]:
# It’s not black magic, you just have to let the wrapper 
# pass the argument:

def a_decorator_passing_arguments(function_to_decorate):
    def a_wrapper_accepting_arguments(arg1, arg2):
        print('I got args! Look:', arg1, arg2)
        function_to_decorate(arg1, arg2)
    return a_wrapper_accepting_arguments

# Since when you are calling the function returned by the decorator, you are
# calling the wrapper, passing arguments to the wrapper will let it pass them to 
# the decorated function

################################## TODO ##################################
# Define function print_full_name and decorate it with the above decorator
@a_decorator_passing_arguments
def print_full_passing_arguments(arg1,arg2):
    print("my name is",arg1,arg2)

print_full_passing_arguments("Peter","Venkman")

# outputs:
#I got args! Look: Peter Venkman
#My name is Peter Venkman

I got args! Look: Peter Venkman
my name is Peter Venkman


#### Decorating methods
If you’re making general-purpose decorator--one you’ll apply to any function or method, no matter its arguments--then just use *args, **kwargs:

In [19]:
def a_decorator_passing_arbitrary_arguments(function_to_decorate):
    # The wrapper accepts any arguments
    def a_wrapper_accepting_arbitrary_arguments(*args, **kwargs):
        print('Do I have args?:')
        print(args)
        print(kwargs)
        # Then you unpack the arguments, here *args, **kwargs
        # If you are not familiar with unpacking, check:
        # http://www.saltycrane.com/blog/2008/01/how-to-use-args-and-kwargs-in-python/
        function_to_decorate(*args, **kwargs)
    return a_wrapper_accepting_arbitrary_arguments

@a_decorator_passing_arbitrary_arguments
def function_with_no_argument():
    print('Python is cool, no argument here.')

function_with_no_argument()
#outputs
#Do I have args?:
#()
#{}
#Python is cool, no argument here.

Do I have args?:
()
{}
Python is cool, no argument here.


In [20]:
################################## TODO ################################## 
# Define function_with_argument and decorate it with a_decorator_passing_arbitrary_arguments
#*는 전체 튜플, **는 dictionary, key와 value로 입력받는다.
@a_decorator_passing_arbitrary_arguments
def function_with_arguments(*args,**kwargs):  
    for i in args:
        print(i,end=' ')

function_with_arguments(1,2,3)

#outputs
#Do I have args?:
#(1, 2, 3)
#{}
#1 2 3 

Do I have args?:
(1, 2, 3)
{}
1 2 3 

In [21]:
################################## TODO ##################################
# Define function_with_named_argument and decorate it with a_decorator_passing_arbitrary_arguments
@a_decorator_passing_arbitrary_arguments
def function_with_named_arguments(*args, **kwargs):
    print("Do",args[0],",",args[1],"and",args[2], "like platypus?", kwargs["platypus"])
    
    
function_with_named_arguments('Bill', 'Linus', 'Steve', platypus='Indeed!')
#outputs
#Do I have args ? :
#('Bill', 'Linus', 'Steve')
#{'platypus': 'Indeed!'}
#Do Bill, Linus and Steve like platypus? Indeed!

Do I have args?:
('Bill', 'Linus', 'Steve')
{'platypus': 'Indeed!'}
Do Bill , Linus and Steve like platypus? Indeed!


#### Passing arguments to the decorator
Great, now what would you say about passing arguments to the decorator itself?

This can get somewhat twisted, since a decorator must accept a function as an argument. Therefore, you cannot pass the decorated function’s arguments directly to the decorator.

Before rushing to the solution, let’s write a little reminder:

In [22]:
#Decorators are ORDINARY functions
def my_decorator(func):
    print('I am an ordinary function')
    def wrapper():
        print('I am function returned by the decorator')
        func()
    return wrapper

#Therefore, you can call it without any '@'

def lazy_function():
    print('zzzzzzzz')

decorated_function = my_decorator(lazy_function)
#outputs: I am an ordinary function
            
#It outputs 'I am an ordinary function', because that’s just what you do:
#calling a function. Nothing magic.

@my_decorator
def lazy_function():
    print('zzzzzzzz')
    
#outputs: I am an ordinary function

I am an ordinary function
I am an ordinary function


It’s exactly the same: my_decorator is called. So when you @my_decorator, you are telling Python to call the function labelled by the variable “my_decorator”.

This is important! The label you give can point directly to the decorator—or not.

Let’s get evil.

In [36]:
def decorator_maker():
    
    print('I make decorators! I am executed only once: '+\
          'when you make me create a decorator.')
            
    def my_decorator(func):
        
        print('I am a decorator! I am executed only when you decorate a function.')
               
        def wrapped():
            print ('I am the wrapper around the decorated function. '
                  'I am called when you call the decorated function. '
                  'As the wrapper, I return the RESULT of the decorated function.')
            return func()
        
        print('As the decorator, I return the wrapped function.')
        
        return wrapped
    
    print('As a decorator maker, I return a decorator')
    return my_decorator

In [32]:
# Let’s create a decorator. It’s just a new function after all.
new_decorator = decorator_maker()   
#outputs:
#I make decorators! I am executed only once: when you make me create a decorator.
#As a decorator maker, I return a decorator

I make decorators! I am executed only once: when you make me create a decorator.
As a decorator maker, I return a decorator


In [33]:
# Then we decorate the function
def decorated_function():
    print('I am the decorated function.')

In [39]:
################################## TODO ##################################->뭔말임
# Define decorated_function
decorated_function=new_decorator(decorated_function)

#outputs:
#I am a decorator! I am executed only when you decorate a function.
#As the decorator, I return the wrapped function

I am a decorator! I am executed only when you decorate a function.
As the decorator, I return the wrapped function.


In [40]:
# Let’s call the function:->뭔말임
decorated_function()
#outputs:
#I am the wrapper around the decorated function. I am called when you call the decorated function.
#As the wrapper, I return the RESULT of the decorated function.
#I am the decorated function.

I am the wrapper around the decorated function. I am called when you call the decorated function. As the wrapper, I return the RESULT of the decorated function.
I am the wrapper around the decorated function. I am called when you call the decorated function. As the wrapper, I return the RESULT of the decorated function.
I am the wrapper around the decorated function. I am called when you call the decorated function. As the wrapper, I return the RESULT of the decorated function.
I am function returned by the decorator
I am the decorated function.


No surprise here.

Let’s do EXACTLY the same thing, but skip all the pesky intermediate variables:

In [41]:
def decorated_function():
    print('I am the decorated function.')

################################## TODO ##################################
decorated_function = decorator_maker()(decorated_function)
#outputs:
#I make decorators! I am executed only once: when you make me create a decorator.
#As a decorator maker, I return a decorator
#I am a decorator! I am executed only when you decorate a function.
#As the decorator, I return the wrapped function.

# Finally:
decorated_function()    
#outputs:
#I am the wrapper around the decorated function. I am called when you call the decorated function.
#As the wrapper, I return the RESULT of the decorated function.
#I am the decorated function.

I make decorators! I am executed only once: when you make me create a decorator.
As a decorator maker, I return a decorator
I am a decorator! I am executed only when you decorate a function.
As the decorator, I return the wrapped function.
I am the wrapper around the decorated function. I am called when you call the decorated function. As the wrapper, I return the RESULT of the decorated function.
I am the decorated function.


Let’s make it even shorter:

In [43]:
################################## TODO ##################################
# Redefine function decorated_function with decorators.
@decorator_maker()
def decorated_function():
    print('I am the decorated function.')
#outputs:
#I make decorators! I am executed only once: when you make me create a decorator.
#As a decorator maker, I return a decorator
#I am a decorator! I am executed only when you decorate a function.
#As the decorator, I return the wrapped function.

#Eventually: 
decorated_function()    
#outputs:
#I am the wrapper around the decorated function. I am called when you call the decorated function.
#As the wrapper, I return the RESULT of the decorated function.
#I am the decorated function.

I make decorators! I am executed only once: when you make me create a decorator.
As a decorator maker, I return a decorator
I am a decorator! I am executed only when you decorate a function.
As the decorator, I return the wrapped function.
I am the wrapper around the decorated function. I am called when you call the decorated function. As the wrapper, I return the RESULT of the decorated function.
I am the decorated function.


### Let’s practice: decorating a decorator
Okay, as a bonus, I’ll give you a snippet to make any decorator accept generically any argument. After all, in order to accept arguments, we created our decorator using another function.

We wrapped the decorator.

Anything else we saw recently that wrapped function?

Oh yes, decorators!

Let’s have some fun and write a decorator for the decorators:

In [44]:
def decorator_with_args(decorator_to_enhance):
    """ 
    This function is supposed to be used as a decorator.
    It must decorate an other function, that is intended to be used as a decorator.
    Take a cup of coffee.
    It will allow any decorator to accept an arbitrary number of arguments,
    saving you the headache to remember how to do that every time.
    """
    
    # We use the same trick we did to pass arguments
    def decorator_maker(*args, **kwargs):
       
        # We create on the fly a decorator that accepts only a function
        # but keeps the passed arguments from the maker.
        def decorator_wrapper(func):
       
            # We return the result of the original decorator, which, after all, 
            # IS JUST AN ORDINARY FUNCTION (which returns a function).
            # Only pitfall: the decorator must have this specific signature or it won’t work:
            return decorator_to_enhance(func, *args, **kwargs)
        
        return decorator_wrapper
    
    return decorator_maker

It can be used as follows:

In [45]:
# You create the function you will use as a decorator. And stick a decorator on it :-)
# Don’t forget, the signature is `decorator(func, *args, **kwargs)`
@decorator_with_args 
def decorated_decorator(func, *args, **kwargs): 
    def wrapper(function_arg1, function_arg2):
        print('Decorated with', args, kwargs)
        return func(function_arg1, function_arg2)
    return wrapper
    
# Then you decorate the functions you wish with your brand new decorated decorator.

################################## TODO ##################################
# Define function decorated_function and decorate this function with decorated_decorator 
# so that the below function call prints the below outputs
@decorated_decorator(42, 404, 1024)
def decorated_function(function_arg1, function_arg2):
    print('Hello', function_arg1, function_arg2)
    
decorated_function('Universe and', 'everything')
#outputs:
#Decorated with (42, 404, 1024) {}
#Hello Universe and everything

# Whoooot!

Decorated with (42, 404, 1024) {}
Hello Universe and everything


### Best practices: decorators

    Decorators were introduced in Python 2.4, so be sure your code will be run on >= 2.4.
    Decorators slow down the function call. Keep that in mind.
    You cannot un-decorate a function. (There are hacks to create decorators that can be removed, but nobody uses them.) So once a function is decorated, it’s decorated for all the code.
    Decorators wrap functions, which can make them hard to debug. (This gets better from Python >= 2.5; see below.)

The functools module was introduced in Python 2.5. It includes the function functools.wraps(), which copies the name, module, and docstring of the decorated function to its wrapper.

(Fun fact: functools.wraps() is a decorator! ☺)

In [ ]:
# For debugging, the stacktrace prints you the function __name__
def foo():
    print('foo')
    
print(foo.__name__)
#outputs: foo
    
# With a decorator, it gets messy    
def bar(func):
    def wrapper():
        print('bar')
        return func()
    return wrapper

@bar
def foo():
    print('foo')

print(foo.__name__)
#outputs: wrapper

# `functools` can help with that

import functools

def bar(func):
    # We say that `wrapper`, is wrapping `func`
    # and the magic begins
    @functools.wraps(func)
    def wrapper():
        print('bar')
        return func()
    return wrapper

@bar
def foo():
    print('foo')

print(foo.__name__)
#outputs: foo

### How can the decorators be useful?
Now the big question: What can I use decorators for?

Seem cool and powerful, but a practical example would be great. Well, there are 1000 possibilities. Classic uses are extending a function behavior from an external lib (you can’t modify it), or for debugging (you don’t want to modify it because it’s temporary).

You can use them to extend several functions in a DRY’s way, like so:

In [67]:
def benchmark(func):
    """
    A decorator that prints the time a function takes
    to execute.
    """
    import time
    import math
    def wrapper(*args, **kwargs):
        ################################## TODO ##################################
        # 1. Measure the runtime of function func.
        # 2. Print the name of func and the runtime
        # 3. Return the return value of func
        s = time.time()
        res = func(*args, **kwargs)
        e = time.time()
        print(func.__name__,e-s)
        
        return res
    
    return wrapper


def logging(func):
    """
    A decorator that logs the activity of the script.
    (it actually just prints it, but it could be logging!)
    """
    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        print(func.__name__, args, kwargs)
        return res
    return wrapper


def counter(func):
    """
    A decorator that counts and prints the number of times a function has been executed
    """
    def wrapper(*args, **kwargs):
        wrapper.count = wrapper.count + 1
        res = func(*args, **kwargs)
        print('{0} has been used: {1}x'.format(func.__name__, wrapper.count))
        return res
    wrapper.count = 0
    return wrapper

In [68]:
################################## TODO ##################################
# Decorate function reverse_string so that the below function calls lead to the below outputs.
@counter
@benchmark
@logging
def reverse_string(string):
    return string[::-1]

print(reverse_string('Able was I ere I saw Elba'))
print(reverse_string('A man, a plan, a canoe, pasta, heros, rajahs, a coloratura, maps, snipe, percale, macaroni, a gag, a banana bag, a tan, a tag, a banana bag again (or a camel), a crepe, pins, Spam, a rut, a Rolo, cash, a jar, sore hats, a peon, a canal: Panama!'))

#outputs:
#reverse_string ('Able was I ere I saw Elba',) {}
#wrapper 0.0
#wrapper has been used: 1x 
#ablE was I ere I saw elbA
#reverse_string ('A man, a plan, a canoe, pasta, heros, rajahs, a coloratura, maps, snipe, percale, macaroni, a gag, a banana bag, a tan, a tag, a banana bag again (or a camel), a crepe, pins, Spam, a rut, a Rolo, cash, a jar, sore hats, a peon, a canal: Panama!',) {}
#wrapper 0.0
#wrapper has been used: 2x
#!amanaP :lanac a ,noep a ,stah eros ,raj a ,hsac ,oloR a ,tur a ,mapS ,snip ,eperc a ,)lemac a ro( niaga gab ananab a ,gat a ,nat a ,gab ananab a ,gag a ,inoracam ,elacrep ,epins ,spam ,arutaroloc a ,shajar ,soreh ,atsap ,eonac a ,nalp a ,nam A

reverse_string ('Able was I ere I saw Elba',) {}
wrapper 0.0
wrapper has been used: 1x
ablE was I ere I saw elbA
reverse_string ('A man, a plan, a canoe, pasta, heros, rajahs, a coloratura, maps, snipe, percale, macaroni, a gag, a banana bag, a tan, a tag, a banana bag again (or a camel), a crepe, pins, Spam, a rut, a Rolo, cash, a jar, sore hats, a peon, a canal: Panama!',) {}
wrapper 0.0
wrapper has been used: 2x
!amanaP :lanac a ,noep a ,stah eros ,raj a ,hsac ,oloR a ,tur a ,mapS ,snip ,eperc a ,)lemac a ro( niaga gab ananab a ,gat a ,nat a ,gab ananab a ,gag a ,inoracam ,elacrep ,epins ,spam ,arutaroloc a ,shajar ,soreh ,atsap ,eonac a ,nalp a ,nam A


Of course the good thing with decorators is that you can use them right away on almost anything without rewriting. DRY, I said:

In [69]:
@counter
@benchmark
@logging
def get_random_futurama_quote():
    from urllib.request import urlopen
    result = urlopen('http://subfusion.net/cgi-bin/quote.pl?quote=futurama').read()
    try:
        value = result.split('<br><b><hr><br>')[1].split('<br><br><hr>')[0]
        return value.strip()
    except:
        return 'No, I’m ... doesn’t!'

    
print(get_random_futurama_quote())
print(get_random_futurama_quote())

#outputs:
#get_random_futurama_quote () {}
#wrapper 0.02
#wrapper has been used: 1x
#The laws of science be a harsh mistress.
#get_random_futurama_quote () {}
#wrapper 0.01
#wrapper has been used: 2x
#Curse you, merciful Poseidon!

URLError: <urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>

### Useful decorators

In [101]:
import pandas as pd

In [102]:
def load_data():  
    df = pd.DataFrame()
    df['height'] = [72.1, 69.8, 63.2, 64.7]
    df['weight'] = [198, 204, 164, 238]
    return df

In [103]:
def mean(data):
    print(data.mean())
def std(data):
    print(data.std())
def minimum(data):
    print(data.min())
def maximum(data):
    print(data.max())

In [104]:
# Add the missing function references to the function map
function_map = {
  'mean': mean,
  'std': std,
  'minimum': minimum,
  'maximum': maximum
}

data = load_data()
print(data)

   height  weight
0    72.1     198
1    69.8     204
2    63.2     164
3    64.7     238


In [105]:
# Call the chosen function and pass "data" as an argument
function_map['mean'](data)
function_map['std'](data)
function_map['minimum'](data)
function_map['maximum'](data)

height     67.45
weight    201.00
dtype: float64
height     4.194043
weight    30.309514
dtype: float64
height     63.2
weight    164.0
dtype: float64
height     72.1
weight    238.0
dtype: float64


In [106]:
def has_docstring(func):
    """Check to see if the function 
  `func` has a docstring.

  Args:
    func (callable): A function.

  Returns:
    bool
  """
    return func.__doc__ is not None

In [107]:
def load_and_plot_data(filename):
    """Load a data frame and plot each column.  
    Args:
    filename (str): Path to a CSV file of data.
    Returns:
    pandas.DataFrame  
    """
    df = pd.load_csv(filename, index_col=0)
    df.hist()
    return df

In [108]:
has_docstring(load_and_plot_data)

True

In [118]:
def create_math_function(func_name):
    if func_name == 'add':
        def add(a, b):
            return a + b
        return add
    elif func_name == 'subtract':
    # Define the subtract() function
        def subtract(a, b):
            return a - b
        return subtract
    else:
        print("I don't know that one")
    
################################## TODO ##################################
# Define variable add with the above function
add = create_math_function('add')
print('5 + 2 = {}'.format(add(5, 2)))

################################## TODO ##################################
# Define variable subtract with the above function
subtract = create_math_function('subtract')
print('5 - 2 = {}'.format(subtract(5, 2)))

5 + 2 = 7
5 - 2 = 3


### Defining a decorator

In [91]:
def print_before_and_after(func):
    def wrapper(*args):
        print('Before {}'.format(func.__name__))
    # Call the function being decorated with *args
        func(*args)
        print('After {}'.format(func.__name__))
  # Return the nested function
    return wrapper

In [92]:
@print_before_and_after
def multiply(a, b):
    print(a * b)

multiply(5, 10)

Before multiply
50
After multiply


In [93]:
## More examples

In [94]:
def print_return_type(func):
    # Define wrapper(), the decorated function
    def wrapper(*args, **kwargs):
        # Call the function being decorated
        result = func(*args, **kwargs)
        print('{}() returned type {}'.format(
          func.__name__, type(result)
        ))
        return result
    # Return the decorated function
    return wrapper

In [95]:
@print_return_type
def foo(value):
    return value
  
print(foo(42))
print(foo([1, 2, 3]))
print(foo({'a': 42}))

foo() returned type <class 'int'>
42
foo() returned type <class 'list'>
[1, 2, 3]
foo() returned type <class 'dict'>
{'a': 42}


In [96]:
def counter(func):
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        # Call the function being decorated and return the result
        return wrapper.count
    wrapper.count = 0
    # Return the new decorated function
    return wrapper

In [100]:
################################## TODO ##################################
# Decorate foo() with the counter() decorator

@counter
def foo(value):
    return value

foo()
foo()
foo()
print('foo() was called {} times.'.format(foo.count))

foo() was called 3 times.


### Decorators with argument

In [84]:
def run_n_times(n):
    """Define and return a decorator"""
    def decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(n):
                func(*args, **kwargs)
        return wrapper
    return decorator

In [85]:
# Make print_sum() run 10 times with the run_n_times() decorator
@run_n_times(10)
def print_sum(a, b):
    print(a + b)

print_sum(15, 20)

35
35
35
35
35
35
35
35
35
35


In [90]:
################################## TODO ##################################
# Use run_n_times() to create the run_five_times() decorator
run_five_times = run_n_times(5)

################################## TODO ##################################
# Decorate print_sum with run_five_times

@run_five_times
def print_sum(a, b):
    print(a + b)
    
print_sum(4, 100)

104
104
104
104
104


In [ ]:
# Modify the print() function to always run 20 times
default_print = print
print = run_n_times(20)(print)


print('What is happening?!?!')
print = default_print
print('I am OK')

### Check the return type

In [70]:
def returns_dict(func):
  # Complete the returns_dict() decorator
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        assert(type(result) == dict)
        return result
    return wrapper

In [71]:
@returns_dict
def foo(value):
    return value

In [72]:
try:
    print(foo([1,2,3]))
except AssertionError:
    print('foo() did not return a dict!')

foo() did not return a dict!


In [73]:
def returns(return_type):
    # Complete the returns() decorator
    def decorator(funct):
        def wrapper(*args, **kwargs):
            result = funct(*args, **kwargs)
            assert(type(result) == return_type)
            return result
        return wrapper
    return decorator
  


In [74]:
@returns(dict)
def foo(value):
    return value


In [75]:
try:
    print(foo([1,2,3]))
except AssertionError:
    print('foo() did not return a dict!')

foo() did not return a dict!


In [82]:
################################## TODO ##################################
# Decorate the below foo function so that there will be no assertion error.
@returns(list)
def foo(value):
    return value

try:
    print(foo([1,2,3]))
except AssertionError:
    print('foo() did not return a dict!')

[1, 2, 3]
